## Preprocess

In [18]:
import pickle, random, re
pfile = open('saved/lyrics_filtered.pkl', 'rb')

lyrics = pickle.load(pfile)
random.shuffle(lyrics)

dev_lyrics = lyrics[:500]

In [20]:
import numpy as np

from gensim.models import Word2Vec

import thulac

In [21]:
class Doc():
    
    model = Word2Vec.load('saved/word2vec_model')
    cut = thulac.thulac(seg_only=True)  #只进行分词，不进行词性标注\n"
    
    corpus = []
    
    def __init__(self, document, tokenizer = 'char'):
        '''Doc class, a representation of document.
        
        @param document: A Chinese sentence.
        @param tokenizer: the tokenizer(char/word)
        '''
        if tokenizer == 'char':
            self.bag_of_words = list(document)
        if tokenizer == 'word':
            self.bag_of_words = list(list(zip(*Doc.cut.cut(document)))[0])
        
        self.vec = self.to_vec()
        Doc.corpus.append(self)
        #Doc.corpus.append(document)
    
    def to_vec(self):
        vec = np.zeros([512,512], dtype='float')
        for word in self.bag_of_words:
            if word in Doc.model.wv:
                vec += Doc.model.wv[word]
            else:
                for char in word:
                    vec += Doc.model.wv[char] if char in Doc.model.wv else 0

        return vec / np.linalg.norm(vec) if np.linalg.norm(vec) > 0 else vec

    def similarity(doc1, doc2):
        '''Return the cosine distance between two sentences.'''
        return np.dot(doc1.vec, doc2.vec)

    def most_similar(self):
        '''Find the most similar sentence in the corpus.

        Similar defined as cosine distance.
        '''
        most_simi, winner = 0, Doc('')

        for i, candidate in enumerate(Doc.corpus):
            simi = Doc.similarity(self, candidate)
            if simi > most_simi and candidate.bag_of_words != self.bag_of_words:
                most_simi, winner = simi, candidate

        return most_simi, ''.join(winner.bag_of_words)
    
    def test():
        '''Unit test & usage'''
        sentence1 = '天青色等烟雨'
        doc1 = Doc(sentence1)
        print('Tokenized and word vec[:10] of %s:' % sentence1)
        print(doc1.bag_of_words)
        print(doc1.to_vec()[:10])
        print('')
        print('Most similar word to 河流')
        print(Doc.model.most_similar('河流'))
        print('')
        sentence2 = '而我在等你'
        doc2 = Doc(sentence2)
        print('Similarity between %s, %s' % (sentence1, sentence2))
        print(Doc.similarity(doc1, doc2))
        print('')
        print('Most similar to %s in corpus' % sentence1)
        print(doc1.most_similar())
        print('')

Model loaded succeed


In [22]:
def load_lyrics(lyrics):
    count = 0
    Doc.corpus = []
    for lyric in lyrics:
        count += 1
        print (count)
        for sentence in lyric:
            Doc(sentence, tokenizer = 'word')
load_lyrics(dev_lyrics)
bag = []
for corpu in Doc.corpus:
    bag.extend(corpu.bag_of_words)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,NMF,LatentDirichletAllocation
from sklearn.cluster import KMeans

In [29]:
# original version
def find_topic(texts, topic_model, n_topics, vec_model="tf", thr=1e-2, **kwargs):
        # 1. vectorization

        vectorizer = CountVectorizer() if vec_model== "tf" else TfidfVectorizer()
        text_vec = vectorizer.fit_transform(texts)
        words = np.array(vectorizer.get_feature_names())

        # 2. topic finding
        topic_models = {"nmf":NMF, "svd": TruncatedSVD, "lda":LatentDirichletAllocation,"kmeans":KMeans}
        topicfinder = topic_models[topic_model](n_topics, **kwargs).fit(text_vec)
        topic_dists = topicfinder.components_ if topic_model is not "kmeans" else topicfinder.cluster_centers_
        topic_dists /= topic_dists.max(axis = 1).reshape((-1,1))
        # 3. keywords for topics
        def _topic_keywords(topic_dist):
            keywords_index = np.abs(topic_dist) >= thr
            keywords_prefix = np.where(np.sign(topic_dist)>0, "","^")[keywords_index]
            keywords = " | ".join(map(lambda x: "".join(x), zip(keywords_prefix, words[keywords_index])))
            return keywords
        topic_keywords = map(_topic_keywords, topic_dists)
        return "\n".join("Topic %i:%s" % (i, t) for i, t in enumerate(topic_keywords))
        
    

In [33]:
# vec model is tf

#print(find_topic(bag,"svd",20, vec_model = "tf"))
#print(find_topic(bag,"nmf",20, vec_model = "tf"))
#print(find_topic(bag,"lda",20, vec_model = "tf"))
#print(find_topic(bag,"kmeans",20, vec_model = "tf"))

# vec model is tfidf

#print(find_topic(bag,"svd",20, vec_model = "tfidf"))
#print(find_topic(bag,"nmf",20, vec_model = "tfidf"))
#print(find_topic(bag,"lda",20, vec_model = "tfidf"))
print(find_topic(bag,"kmeans",20, vec_model = "tfidf"))

Topic 0:一万 | 一下 | 一丝丝 | 一二三四 | 一些 | 一再 | 一刻 | 一千 | 一千一千杯苦 | 一千万 | 一半 | 一句句 | 一同 | 一声声 | 一天天 | 一定 | 一帘 | 一幕幕 | 一度 | 一张张 | 一心一意 | 一息 | 一整天 | 一整晚 | 一旁 | 一旦 | 一时 | 一朝 | 一样 | 一次次 | 一步步 | 一点 | 一点一滴 | 一点一点 | 一点点 | 一生 | 一生一世 | 一生有 | 一番 | 一百 | 一直 | 一瞬间 | 一绪手 | 一群群 | 一行 | 一见钟情 | 一起 | 一路 | 一路多 | 一身 | 一转眼 | 一辈子 | 一边 | 一遍遍 | 一阵 | 一阵阵 | 一齐 | 丁小毛 | 丁思忖 | 万万 | 万众 | 万岁 | 万沛鑫 | 万物 | 万芳 | 三五 | 三亚湾 | 三日 | 三生有缘 | 三百 | 上学 | 上帝 | 上弦月 | 上档次 | 上次 | 上班 | 上门 | 上面 | 下去 | 下地 | 下落 | 下落不明 | 下诉 | 下车 | 下辈子 | 下雨 | 下雨天 | 下雪 | 下面 | 不堪设想 | 不知不觉 | 不翼而飞 | 不见不散 | 不言不语 | 不顾一切 | 与众不同 | 专利 | 专心 | 专辑 | 专辑原 | 世上 | 世事 | 世人 | 世俗 | 世外 | 世态炎凉 | 世痛 | 世纪 | 世间 | 东京 | 东水 | 东海 | 东西 | 东风 | 丝丝 | 両手 | 丢失 | 丢掉 | 两三 | 两地 | 两手空空 | 两旁 | 两端 | 个个 | 个人 | 中作弄 | 中华 | 中国 | 中央 | 中岛 | 中心 | 中文 | 中间 | 丰富 | 临别 | 为什 | 为何 | 为着 | 为难 | 主人 | 主唱 | 主场 | 主宰 | 主意 | 主要 | 主角 | 主题 | 主题曲 | 举起 | 乃至 | 久久 | 久别 | 义无反顾 | 之一 | 之上 | 之下 | 之中 | 之内 | 之前 | 之后 | 之外 | 之间 | 之际 | 乌云 | 乌有 | 乌龟 | 乏味 | 乐团 | 乐声 | 乐天 | 乐意 | 乐此不疲 | 乐逍 | 乐队 | 乖乖 | 乘着 | 九十 | 九州 | 九月 | 习习 | 习惯 | 习惯成自然 | 乡头 | 

In [ ]:
#test_texts = np.repeat(['男人爱吃汉堡', '女人不爱吃汉堡'],[100,100])
test_texts = [['男人','爱','吃','汉堡', '女人','不爱','吃','薯条'],['男人','爱','吃','汉堡', '女人','不爱','吃','薯条']]
#test_texts = np.repeat(['男人','爱','吃','汉堡', '女人','不爱','吃','汉堡'],[100,100])
#test_texts = np.repeat(['man like burgers', 'women dislike burgers'],[100,100])


In [2]:
lyrics.shape

NameError: name 'lyrics' is not defined